In [ ]:
! pip install spopt
! pip install pulp

class MCLP(LocateSolver, BaseOutputMixin, CoveragePercentageMixin):
    r"""
    Implement the Maximal Coverage Location Problem (*MCLP*) optimization model
    and solve it. The *MCLP*, as adapted from :cite:`church_murray_2018`,
    can be formulated as:

    .. math::

       \begin{array}{lllll}
       \displaystyle \textbf{Maximize}      & \displaystyle \sum_{i \in I}{a_iX_i}          &&                  & (1)                                                                                   \\
       \displaystyle \textbf{Subject To}    & \displaystyle \sum_{j \in N_i}{Y_j \geq X_i}  && \forall i \in I  & (2)                                                                                   \\
                                            & \displaystyle \sum_{j \in J}{Y_j} = p         &&                  & (3)                                                                                   \\
                                            & X_i \in \{0, 1\}                              && \forall i \in I  & (4)                                                                                   \\
                                            & Y_j \in \{0, 1\}                              && \forall j \in J  & (5)                                                                                   \\
                                            &                                               &&                  &                                                                                       \\
       \displaystyle \textbf{Where}         && i                                            & =                 & \textrm{index of demand points/areas/objects in set } I                               \\
                                            && j                                            & =                 & \textrm{index of potential facility sites in set } J                                  \\
                                            && p                                            & =                 & \textrm{the number of facilities to be sited}                                         \\
                                            && S                                            & =                 & \textrm{maximum acceptable service distance or time standard}                         \\
                                            && d_{ij}                                       & =                 & \textrm{shortest distance or travel time between locations } i \textrm{ and } j       \\
                                            && N_i                                          & =                 & \{j | d_{ij} < S\}                                                                    \\
                                            && X_i                                          & =                 & \begin{cases}
                                                                                                                   1, \textrm{if client location } i \textrm{ is covered within service standard } S    \\
                                                                                                                   0, \textrm{otherwise}                                                                \\
                                                                                                                  \end{cases}                                                                           \\
                                            && Y_j                                          & =                 & \begin{cases}
                                                                                                                   1, \textrm{if a facility is sited at location } j                                    \\
                                                                                                                   0, \textrm{otherwise}                                                                \\
                                                                                                                  \end{cases}
       \end{array}

    Parameters
    ----------

    name : str
        The problem name.
    problem : pulp.LpProblem
        A ``pulp`` instance of an optimization model that contains
        constraints, variables, and an objective function.

    Attributes
    ----------

    name : str
        The problem name.
    problem : pulp.LpProblem
        A ``pulp`` instance of an optimization model that contains
        constraints, variables, and an objective function.
    fac2cli : numpy.array
        A 2D array storing facility to client relationships where each
        row represents a facility and contains an array of client indices
        with which it is associated. An empty client array indicates
        the facility is associated with no clients.
    cli2fac : numpy.array
        The inverse of ``fac2cli`` where client to facility relationships
        are shown.
    aij : numpy.array
        A cost matrix in the form of a 2D array between origins and destinations.
    n_cli_uncov : int
        The number of uncovered client locations.

    """  # noqa: E501

    def __init__(self, name: str, problem: pulp.LpProblem):
        super().__init__(name, problem)

    def __add_obj(self, weights: np.array, range_clients: range) -> None:
        """
        Add the objective function to the model.

        Maximize w1 * y1 + w2 * y2 +  ... + wi * yi

        Returns
        -------

        None

        """
        dem_vars = getattr(self, "cli_vars")

        self.problem += (
            pulp.lpSum([weights.flatten()[i] * dem_vars[i] for i in range_clients]),
            "objective function",
        )

    @classmethod
    def from_cost_matrix(
        cls,
        cost_matrix: np.array,
        weights: np.array,
        service_radius: float,
        p_facilities: int,
        predefined_facilities_arr: np.array = None,
        name: str = "mclp",
    ):
        """
        Create a ``MCLP`` object based on cost matrix.

        Parameters
        ----------

        cost_matrix : numpy.array
            A cost matrix in the form of a 2D array between origins and destinations.
        weights : numpy.array
            A 1D array of service load or population demand.
        service_radius : float
            Maximum acceptable service distance.
        p_facilities : int
            The number of facilities to be located.
        predefined_facilities_arr : numpy.array (default None)
            A binary 1D array of service facilities that must appear in the
            solution. For example, consider 3 facilites ``['A', 'B', 'C']``.
            If facility ``'B'`` must be in the model solution, then the passed
            in array should be ``[0, 1, 0]``.
        name : str (default 'mclp')
            The problem name.

        Returns
        -------

        spopt.locate.coverage.MCLP

        Examples
        --------

        >>> from spopt.locate import MCLP
        >>> from spopt.locate.util import simulated_geo_points
        >>> import geopandas
        >>> import numpy
        >>> import pulp
        >>> import spaghetti

        Create a regular lattice.

        >>> lattice = spaghetti.regular_lattice((0, 0, 10, 10), 9, exterior=True)
        >>> ntw = spaghetti.Network(in_data=lattice)
        >>> streets = spaghetti.element_as_gdf(ntw, arcs=True)
        >>> streets_buffered = geopandas.GeoDataFrame(
        ...     geopandas.GeoSeries(streets["geometry"].buffer(0.2).unary_union),
        ...     crs=streets.crs,
        ...     columns=["geometry"]
        ... )

        Simulate points about the lattice.

        >>> demand_points = simulated_geo_points(streets_buffered, needed=100, seed=5)
        >>> facility_points = simulated_geo_points(streets_buffered, needed=5, seed=6)

        Snap the points to the network of lattice edges.

        >>> ntw.snapobservations(demand_points, "clients", attribute=True)
        >>> clients_snapped = spaghetti.element_as_gdf(
        ...     ntw, pp_name="clients", snapped=True
        ... )
        >>> ntw.snapobservations(facility_points, "facilities", attribute=True)
        >>> facilities_snapped = spaghetti.element_as_gdf(
        ...     ntw, pp_name="facilities", snapped=True
        ... )

        Calculate the cost matrix from origins to destinations.

        >>> cost_matrix = ntw.allneighbordistances(
        ...    sourcepattern=ntw.pointpatterns["clients"],
        ...    destpattern=ntw.pointpatterns["facilities"]
        ... )

        Simulate demand weights from ``1`` to ``12``.

        >>> ai = numpy.random.randint(1, 12, 100)

        Create and solve an ``MCLP`` instance from the cost matrix.

        >>> mclp_from_cost_matrix = MCLP.from_cost_matrix(
        ...     cost_matrix, ai, service_radius=7, p_facilities=4
        ... )
        >>> mclp_from_cost_matrix = mclp_from_cost_matrix.solve(
        ...     pulp.PULP_CBC_CMD(msg=False)
        ... )

        Get the facility lookup demand coverage array.

        >>> for fac, cli in enumerate(mclp_from_cost_matrix.fac2cli):
        ...     print(f"facility {fac} serving {len(cli)} clients")
        facility 0 serving 44 clients
        facility 1 serving 54 clients
        facility 2 serving 75 clients
        facility 3 serving 77 clients
        facility 4 serving 0 clients

        Get the number of clients uncovered and percentage covered.

        >>> mclp_from_cost_matrix.n_cli_uncov
        1
        >>> mclp_from_cost_matrix.perc_cov
        99.0

        """

Number of AEDS:
- Number of AEDs in Brussels: 2024 
- Number of AEDs in Antwerpen: 722
- Number of AEDs in Gent: 389
- Number of AEDs in Charleroi: 365
- Number of AEDs in Liege: 686
- Number of AEDs in Brugge: 161
- Number of AEDs in Leuven: 167
- Number of AEDs in Oostende: 69

In [1]:
from spopt.locate import MCLP
import pandas as pd
import numpy as np
import pulp
import os



In [6]:
##Both the cost matrices and combined mandatory lists need to 
# be imported and then converted to numpy arrays

import_folder_path = "/Users/sammcmanagan/Library/Mobile Documents/com~apple~CloudDocs/Documents/M.Sc Statistics & Data Science/Modern Data Analytics/MDA-Project/Data"

cities = ["Antwerpen", "Brugge", "Brussels", "Charleroi", "Gent", "Leuven", "Liege", "Oostende"]

# Dictionaries to hold the data
cost_matrices = {}
predefined_lists = {}
weights = {}

for city in cities:
    # Import data on the city
    os.chdir(import_folder_path)
    
    # Read the CSV files into DataFrames
    cost_matrix_df = pd.read_csv(city + "_cost_matrix.csv", index_col=0)
    predefined_list_df = pd.read_csv(city + "_mandatory_array.csv")
    
    # Convert the DataFrames to NumPy arrays
    cost_matrix_np = cost_matrix_df.to_numpy()
    predefined_list_np = predefined_list_df.to_numpy().flatten()
    weights_np = np.ones(cost_matrix_np.shape[0])
    
    # Store the NumPy arrays in dictionaries with city names as keys
    cost_matrices[city] = cost_matrix_np
    predefined_lists[city] = predefined_list_np
    weights[city] = weights_np

# Check the results
for city in cities:
    # print(f"{city} Cost Matrix:\n", cost_matrices[city])
    # print(f"{city} Predefined List:\n", predefined_lists[city])
    # print("\n")
    print(len(predefined_lists[city]))
    print(cost_matrices[city].shape)

2488
(1733, 2488)
1257
(405, 1257)
4866
(3968, 4866)
1396
(1067, 1396)
1877
(950, 1877)
827
(318, 827)
2548
(1528, 2548)
459
(297, 459)


In [34]:
##Initiate the class
Oostende_mclp = MCLP.from_cost_matrix(cost_matrix=cost_matrices['Oostende'], ##to be computed for each city in euclidean distance matrix.ipynb and read in, needs to be np array
                                      predefined_facilities_arr= predefined_lists['Oostende'], ##read in from data folder, also needs to be nparray
                                      weights= weights['Oostende'],
                                      p_facilities=69, ##number of aeds in solution, set according to number of aeds in corresponding city
                                      service_radius=150) ##coverage radius in meters, considering all of the distances I've seen so far are over 100m, prob best to set it between 150-200


##Run the algorithm, solving it using pulp optimizer
Oostende_mclp = Oostende_mclp.solve(pulp.PULP_CBC_CMD(msg=False))



##Analyzing results
print("Number of uncovered cards:",Oostende_mclp.n_cli_uncov)
print("Percentage of cards covered:",Oostende_mclp.perc_cov)


# Creating a df of which AEDs were selected
facility_status = []

# Use the correct attribute to get the facility variables
for facility_index, variable in enumerate(Oostende_mclp.fac_vars):
    status = "selected" if variable.varValue == 1 else "not selected"
    facility_status.append([facility_index, status])


##List of which aed locs were selected, corresponds to col index of cost matrix/row index of combined_aeds_coordinates
Oostende_selected = pd.DataFrame(facility_status, columns=['Facility Index', 'Selection Status'])

##To verify cost matrix is being read correctly,
# len of Oostende_selected is 459, which corresponds to n of possible 
# aed locs, so it is working correctly
print(len(Oostende_selected))

Number of uncovered cards: 158
Percentage of cards covered: 46.801346801346796
459


In [35]:
##Initiate the class
Leuven_mclp = MCLP.from_cost_matrix(cost_matrix=cost_matrices['Leuven'], ##to be computed for each city in euclidean distance matrix.ipynb and read in, needs to be np array
                                      predefined_facilities_arr= predefined_lists['Leuven'], ##read in from data folder, also needs to be nparray
                                      weights= weights['Leuven'],
                                      p_facilities=167, ##number of aeds in solution, set according to number of aeds in corresponding city
                                      service_radius=150) ##coverage radius in meters, considering all of the distances I've seen so far are over 100m, prob best to set it between 150-200


##Run the algorithm
Leuven_mclp = Leuven_mclp.solve(pulp.PULP_CBC_CMD(msg=False))



##Analyzing results
print("Number of uncovered cards:",Leuven_mclp.n_cli_uncov)
print("Percentage of cards covered:",Leuven_mclp.perc_cov)


# Creating a df of which AEDs were selected
facility_status = []

# Use the correct attribute to get the facility variables
for facility_index, variable in enumerate(Leuven_mclp.fac_vars):
    status = "selected" if variable.varValue == 1 else "not selected"
    facility_status.append([facility_index, status])

Leuven_selected = pd.DataFrame(facility_status, columns=['Facility Index', 'Selection Status'])

Number of uncovered cards: 155
Percentage of cards covered: 51.257861635220124


In [36]:
##Initiate the class
Brugge_mclp = MCLP.from_cost_matrix(cost_matrix=cost_matrices['Brugge'], ##to be computed for each city in euclidean distance matrix.ipynb and read in, needs to be np array
                                      predefined_facilities_arr= predefined_lists['Brugge'], ##read in from data folder, also needs to be nparray
                                      weights= weights['Brugge'],
                                      p_facilities=167, ##number of aeds in solution, set according to number of aeds in corresponding city
                                      service_radius=150) ##coverage radius in meters, considering all of the distances I've seen so far are over 100m, prob best to set it between 150-200


##Run the algorithm
Brugge_mclp = Brugge_mclp.solve(pulp.PULP_CBC_CMD(msg=False))



##Analyzing results
print("Number of uncovered cards:",Brugge_mclp.n_cli_uncov)
print("Percentage of cards covered:",Brugge_mclp.perc_cov)


# Creating a df of which AEDs were selected
facility_status = []

# Use the correct attribute to get the facility variables
for facility_index, variable in enumerate(Brugge_mclp.fac_vars):
    status = "selected" if variable.varValue == 1 else "not selected"
    facility_status.append([facility_index, status])

Brugge_selected = pd.DataFrame(facility_status, columns=['Facility Index', 'Selection Status'])

Number of uncovered cards: 232
Percentage of cards covered: 42.71604938271605


In [37]:
##Initiate the class
Antwerpen_mclp = MCLP.from_cost_matrix(cost_matrix=cost_matrices['Antwerpen'], ##to be computed for each city in euclidean distance matrix.ipynb and read in, needs to be np array
                                      predefined_facilities_arr= predefined_lists['Antwerpen'], ##read in from data folder, also needs to be nparray
                                      weights= weights['Antwerpen'],
                                      p_facilities=722, ##number of aeds in solution, set according to number of aeds in corresponding city
                                      service_radius=150) ##coverage radius in meters, considering all of the distances I've seen so far are over 100m, prob best to set it between 150-200


##Run the algorithm
Antwerpen_mclp = Antwerpen_mclp.solve(pulp.PULP_CBC_CMD(msg=False))



##Analyzing results
print("Number of uncovered cards:",Antwerpen_mclp.n_cli_uncov)
print("Percentage of cards covered:",Antwerpen_mclp.perc_cov)


# Creating a df of which AEDs were selected
facility_status = []

# Use the correct attribute to get the facility variables
for facility_index, variable in enumerate(Antwerpen_mclp.fac_vars):
    status = "selected" if variable.varValue == 1 else "not selected"
    facility_status.append([facility_index, status])

Antwerpen_selected = pd.DataFrame(facility_status, columns=['Facility Index', 'Selection Status'])

Number of uncovered cards: 754
Percentage of cards covered: 56.49163300634738


In [38]:
##Initiate the class
Gent_mclp = MCLP.from_cost_matrix(cost_matrix=cost_matrices['Gent'], ##to be computed for each city in euclidean distance matrix.ipynb and read in, needs to be np array
                                      predefined_facilities_arr= predefined_lists['Gent'], ##read in from data folder, also needs to be nparray
                                      weights= weights['Gent'],
                                      p_facilities=389, ##number of aeds in solution, set according to number of aeds in corresponding city
                                      service_radius=150) ##coverage radius in meters, considering all of the distances I've seen so far are over 100m, prob best to set it between 150-200


##Run the algorithm
Gent_mclp = Gent_mclp.solve(pulp.PULP_CBC_CMD(msg=False))



##Analyzing results
print("Number of uncovered cards:",Gent_mclp.n_cli_uncov)
print("Percentage of cards covered:",Gent_mclp.perc_cov)


# Creating a df of which AEDs were selected
facility_status = []

# Use the correct attribute to get the facility variables
for facility_index, variable in enumerate(Gent_mclp.fac_vars):
    status = "selected" if variable.varValue == 1 else "not selected"
    facility_status.append([facility_index, status])

Gent_selected = pd.DataFrame(facility_status, columns=['Facility Index', 'Selection Status'])

Number of uncovered cards: 461
Percentage of cards covered: 51.473684210526315


In [39]:
##Initiate the class
Brussels_mclp = MCLP.from_cost_matrix(cost_matrix=cost_matrices['Brussels'], ##to be computed for each city in euclidean distance matrix.ipynb and read in, needs to be np array
                                      predefined_facilities_arr= predefined_lists['Brussels'], ##read in from data folder, also needs to be nparray
                                      weights= weights['Brussels'],
                                      p_facilities=2024, ##number of aeds in solution, set according to number of aeds in corresponding city
                                      service_radius=150) ##coverage radius in meters, considering all of the distances I've seen so far are over 100m, prob best to set it between 150-200


##Run the algorithm
Brussels_mclp = Brussels_mclp.solve(pulp.PULP_CBC_CMD(msg=False))



##Analyzing results
print("Number of uncovered cards:",Brussels_mclp.n_cli_uncov)
print("Percentage of cards covered:",Brussels_mclp.perc_cov)


# Creating a df of which AEDs were selected
facility_status = []

# Use the correct attribute to get the facility variables
for facility_index, variable in enumerate(Brussels_mclp.fac_vars):
    status = "selected" if variable.varValue == 1 else "not selected"
    facility_status.append([facility_index, status])

Brussels_selected = pd.DataFrame(facility_status, columns=['Facility Index', 'Selection Status'])

Number of uncovered cards: 1075
Percentage of cards covered: 72.90826612903226


In [41]:
##Initiate the class
Liege_mclp = MCLP.from_cost_matrix(cost_matrix=cost_matrices['Liege'], ##to be computed for each city in euclidean distance matrix.ipynb and read in, needs to be np array
                                      predefined_facilities_arr= predefined_lists['Liege'], ##read in from data folder, also needs to be nparray
                                      weights= weights['Liege'],
                                      p_facilities=686, ##number of aeds in solution, set according to number of aeds in corresponding city
                                      service_radius=150) ##coverage radius in meters, considering all of the distances I've seen so far are over 100m, prob best to set it between 150-200


##Run the algorithm
Liege_mclp = Liege_mclp.solve(pulp.PULP_CBC_CMD(msg=False))



##Analyzing results
print("Number of uncovered cards:",Liege_mclp.n_cli_uncov)
print("Percentage of cards covered:",Liege_mclp.perc_cov)


# Creating a df of which AEDs were selected
facility_status = []

# Use the correct attribute to get the facility variables
for facility_index, variable in enumerate(Liege_mclp.fac_vars):
    status = "selected" if variable.varValue == 1 else "not selected"
    facility_status.append([facility_index, status])

Liege_selected = pd.DataFrame(facility_status, columns=['Facility Index', 'Selection Status'])

Number of uncovered cards: 793
Percentage of cards covered: 48.1020942408377


In [42]:
##Initiate the class
Charleroi_mclp = MCLP.from_cost_matrix(cost_matrix=cost_matrices['Charleroi'], ##to be computed for each city in euclidean distance matrix.ipynb and read in, needs to be np array
                                      predefined_facilities_arr= predefined_lists['Charleroi'], ##read in from data folder, also needs to be nparray
                                      weights= weights['Charleroi'],
                                      p_facilities=365, ##number of aeds in solution, set according to number of aeds in corresponding city
                                      service_radius=150) ##coverage radius in meters, considering all of the distances I've seen so far are over 100m, prob best to set it between 150-200


##Run the algorithm
Charleroi_mclp = Charleroi_mclp.solve(pulp.PULP_CBC_CMD(msg=False))



##Analyzing results
print("Number of uncovered cards:",Charleroi_mclp.n_cli_uncov)
print("Percentage of cards covered:",Charleroi_mclp.perc_cov)


# Creating a df of which AEDs were selected
facility_status = []

# Use the correct attribute to get the facility variables
for facility_index, variable in enumerate(Charleroi_mclp.fac_vars):
    status = "selected" if variable.varValue == 1 else "not selected"
    facility_status.append([facility_index, status])

Charleroi_selected = pd.DataFrame(facility_status, columns=['Facility Index', 'Selection Status'])

Number of uncovered cards: 653
Percentage of cards covered: 38.80037488284911
